
## 2. Implementación Metodológica Principal (5-7 páginas + código)

El contenido específico dependerá del enfoque elegido, pero TODOS deben incluir:

### Sección A: Preparación y Transformación

#### Proceso detallado de limpieza y transformación de datos

In [ ]:
# Leer los datos
import pandas as pd
import numpy as np

df = pd.read_excel("../data/raw/Viajes Sep-Dic 24 v2.xlsx", sheet_name='Viajes')

Feature: Distancia recorrida en KM

In [ ]:
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import time

def calcular_distancia_recta(row):
    try:
        geolocator = Nominatim(user_agent="mi_app_geo") # Define tu user_agent
        
        # Concatena tus campos para darle más contexto a Nominatim
        query_origen = f"{row['Desc Org. Apt']}, {row['Origen']}"
        query_destino = f"{row['Destino']}"

        # Geocodifica el origen
        location_origen = geolocator.geocode(query_origen)
        time.sleep(1) # IMPORTANTE: Nominatim requiere un delay de 1 seg por petición
        
        # Geocodifica el destino
        location_destino = geolocator.geocode(query_destino)
        time.sleep(1)

        if location_origen and location_destino:
            coords_origen = (location_origen.latitude, location_origen.longitude)
            coords_destino = (location_destino.latitude, location_destino.longitude)
            
            # Calcula la distancia geodésica (línea recta)
            distancia_km = geodesic(coords_origen, coords_destino).kilometers
            print(f"Éxito: {query_origen} -> {query_destino} = {distancia_km:.2f} km")
            return distancia_km
        else:
            print(f"No se pudieron geocodificar: {query_origen} o {query_destino}")
            return np.nan

    except Exception as e:
        print(f"Error en Nominatim: {e}")
        return np.nan

In [ ]:
# Modificamos el DF para estandarizar nombres de Origen y Destino
df['Desc Org. Apt'] = df['Origen'].replace('Churubusco', 'Churubusco Monterrey')
df['Destino'] = df['Destino'].replace('AM Monterrey Plantas', 'Monterrey')
df['Destino'] = df['Destino'].replace('AM Monterrey Local', 'Monterrey')
df['Destino'] = df['Destino'].replace('Area Metr Queretaro', 'Queretaro')
df['Destino'] = df['Destino'].replace('Área Metro Saltillo', 'Saltillo')
df['Destino'] = df['Destino'].replace('Area Metro Monclova', 'Monclova')
df['Destino'] = df['Destino'].replace('Área Metro Monclova', 'Monclova')
df['Destino'] = df['Destino'].replace('Área Metro SLP', 'San Luis Potosi')
df['Destino'] = df['Destino'].replace('Área Metro Puebla', 'Puebla')
df['Destino'] = df['Destino'].replace('Área Metro Celaya', 'Celaya')
df['Destino'] = df['Destino'].replace('A. M. Aguascalientes', 'Aguascalientes')

# Creamos un dataframe con las combinaciones únicas de Origen y Destino
combinaciones = df[['Desc Org. Apt','Origen', 'Destino']].drop_duplicates()

# Obtenemos la distancia entre cada par de Origen y Destino

# Primero verificamos si el archivo ya existe para evitar recalcular
import os
if os.path.exists('../data/processed/combinaciones_origen_destino.csv'):
    combinaciones = pd.read_csv('../data/processed/combinaciones_origen_destino.csv')
else:
    combinaciones['Distancia_km'] = combinaciones.apply(calcular_distancia_recta, axis=1)

# Añadimos la distancia al dataframe original
df = df.merge(combinaciones[['Desc Org. Apt','Origen', 'Destino', 'Distancia_km']], on=['Desc Org. Apt','Origen', 'Destino'], how='left')

# Guardamos las combinaciones en un archivo CSV
combinaciones.to_csv('../data/processed/combinaciones_origen_destino.csv', index=False)

#print(combinaciones)

Features: Tipo Planta

In [ ]:
# One Hot Encoding para la columna 'Tipo Planta'
tipo_planta_dummies = pd.get_dummies(df['Tipo Planta'], prefix='Tipo_Planta')
df = pd.concat([df, tipo_planta_dummies], axis=1)

# Eliminar variable original
df.drop('Tipo Planta', axis=1, inplace=True)

Features: Depósito Origen

In [ ]:
# one hot encoding para la columna 'Depósito Origen'
deposito_origen_dummies = pd.get_dummies(df['Deposito Origen'], prefix='Deposito_Origen')
df = pd.concat([df, deposito_origen_dummies], axis=1)

# Eliminar variable original
df.drop('Deposito Origen', axis=1, inplace=True)

Features: Tipo de permiso

In [ ]:
# One hot encoding para la columna 'Tipo de permiso'
tipo_permiso_dummies = pd.get_dummies(df['TpoPermiso'], prefix='Tipo_Permiso')
df = pd.concat([df, tipo_permiso_dummies], axis=1)

# Eliminar variable original
df.drop('TpoPermiso', axis=1, inplace=True)

Features: Tipo de Servicio

# One hot encoding para la columna 'Tipo Servicio'
tipo_servicio_dummies = pd.get_dummies(df['TpoSrv'], prefix='Tipo_Servicio')
df = pd.concat([df, tipo_servicio_dummies], axis=1)

# Eliminar variable original
df.drop('TpoSrv', axis=1, inplace=True)

Features: Tipo de camion

In [ ]:
# Feature: 'Ejes'
def obtener_ejes(tipo_camion):
    if pd.isnull(tipo_camion):
        return 0
    elif tipo_camion == 'Big Coil':
        return 3
    elif tipo_camion == '3 Ejes':
        return 3
    elif tipo_camion == '3 Eje Cortina':
        return 3
    elif tipo_camion == '3 ejes Portacinta':
        return 3
    elif tipo_camion == "Plataforma 48'" or tipo_camion == "Plataforma 53'":
        return 4
    elif tipo_camion == "2 Ejes":
        return 2

df['Ejes'] = df['Tipo transporte'].apply(obtener_ejes)

Feature: Tipo de viaje

In [ ]:
# Primero se añade 'Va' a los valores vacios
df['TpoVje'] = df['TpoVje'].fillna('Vacio')

# One hot encoding para la columna 'TpoVje'
tipo_viaje_dummies = pd.get_dummies(df['TpoVje'], prefix='Tipo_Viaje')
df = pd.concat([df, tipo_viaje_dummies], axis=1)
# Eliminar variable original
df.drop('TpoVje', axis=1, inplace=True)


Feature: Fecha de salida

In [ ]:
# Dividir la columna 'F.Salida' en componentes separados
df['F.Salida'] = pd.to_datetime(df['F.Salida'], errors='coerce')
df['Salida_Año'] = df['F.Salida'].dt.year
df['Salida_Mes'] = df['F.Salida'].dt.month
df['Salida_Dia'] = df['F.Salida'].dt.day

# Eliminamos la columna original
df.drop('F.Salida', axis=1, inplace=True)

In [ ]:
# Modificamos columnas que puedan tener acentos
df.rename(columns={
    'Garantía': 'Garantia',
    'Variación': 'Variacion'
}, inplace=True)

Eliminacion de datos

In [ ]:
# Limpieza de datos
df = df.drop(columns=['Viaje', 'Permiso', 'Shipment', 'Sociedad','Ori-Dest-TT','Ori-Dest','Modal.','Moneda','Estatus'])

df = df.drop(columns=['Planta Origen', 'Origen', 'Destino', 'Desc Org. Apt', 'TPP', 'Desc TPP', 'Transp.Leg'])

df = df.drop(columns=['Tipo planta', 'Nombre', 'Desc TpoTrn', 'Tipo transporte', 'TpoTrn.APT'])

df = df.drop(columns=['TpoTrn.Leg'])

# Se elimina la última linea que contiene totales
df = df.iloc[:-1]

df.head()

# Guardar el dataframe procesado
df.to_csv("../data/processed/datos_procesados.csv", index=False)



#### Justificación de decisiones sobre valores faltantes, outliers, y agregaciones

| Variable a eliminar | Razón |
| ---- | ---- |
| Viaje | Información irrelevante |
| Permiso | Información irrelevante |
| Shipment | Información irrelevante |
| Sociedad | Información irrelevante |
| Ori-Dest-TT | Información redundante |
| Ori-Dest | Información redundante |
| Modal | Información irrelevante |
| Moneda | Información irrelevante |
| Estatus | Información irrelevante |

#### Creación de variables relevantes para su enfoque (ratios, interacciones, lags, etc.)
#### Validación de supuestos requeridos por sus métodos
 

⠀Sección B: Análisis Principal (varía según enfoque)

- Construcción y justificación del DAG (Directed Acyclic Graph)
- Identificación de confounders, mediadores, y colisionadores
- Estrategia de identificación causal (IV, diferencias-en-diferencias, etc.)
- Estimación de efectos causales con intervalos de confianza
- Análisis de sensibilidad a violaciones de supuestos
 

# Construccion del DAG

1. Importamos todas las librerías necesarias

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from statsmodels.api import OLS, add_constant

# Try multiple possible import paths for causal-learn / causallearn to support different package versions
try:
    # Preferred package layout (causal-learn)
    from causal_learn.search.ConstraintBased.PC import pc
    from causal_learn.utils.cit import fisherz
except Exception:
    try:
        # Alternative package name / layout (causallearn)
        from causallearn.search.ConstraintBased.PC import pc
        from causallearn.utils.cit import fisherz
    except Exception as e:
        raise ImportError(
            "Could not import 'pc' and 'fisherz' from causal-learn/causallearn; "
            "please install 'causal-learn' (pip install causal-learn) "
            "and verify the package version and import paths."
        ) from e

2. Configuraciones y constantes

In [ ]:
FILE_PATH = "../data/processed/datos_procesados.csv"  # IMPORTANTE: Reemplaza con tu ruta de archivo
TARGET_VARIABLE_NAME = 'Costo'                 
COEFFICIENT_THRESHOLD = 0.05                   
SELECTED_COLUMNS = [
    'Garantia', 
    'Flete Falso (KG)', 
    'Flete Falso (MXN)', 
    #'Flete Falso (USD)', ELIMINADO POR REDUNDANCIA
    'H.Salida',
    #'#Remitos', ELIMINADO POR REDUNDANCIA
    'Peso Total (kg)',
    'Costo',
    'Costo Prom',
    'Variacion',
    #'CostoxTn',
    'Shp.Cost',
    'Monto Real',
    #'Monto Falso',
    'Monto Reparto',
    'Distancia_km',
    'Tipo_Planta_Masivos',
    #'Tipo_Planta_Revestidos',
    'Deposito_Origen_CHUCS',
    'Deposito_Origen_CHUMA',
    'Deposito_Origen_MVAMI',
    'Deposito_Origen_MVAML',
    #'Deposito_Origen_UNIUN',
    'Tipo_Permiso_1.0',
    'Tipo_Permiso_2.0',
    #'Tipo_Permiso_3.0',
    'Tipo_Servicio_EX',
    #'Tipo_Servicio_FO',
    'Ejes',
    'Tipo_Viaje_AP',
    'Tipo_Viaje_EX',
    'Tipo_Viaje_LT',
    'Tipo_Viaje_NO',
    #'Tipo_Viaje_PM',
    #'Tipo_Viaje_Vacio',
    #'Salida_Año', Eliminada porque todos los datos son del 2024
    'Salida_Mes',
    'Salida_Dia'
]

3. Función de visualización para PC

In [ ]:
def plot_pc_dag(adj_matrix, columns, causal_strengths=None, title='Grafo Causal (CPDAG) Algoritmo PC'):
    """Visualiza el grafo de estructura (CPDAG) generado por el Algoritmo PC."""
    G = nx.DiGraph()
    G.add_nodes_from(columns)
    edge_labels = {}
    
    # Mapear la matriz de adyacencia de PC a un grafo NetworkX
    for i, col_i in enumerate(columns):
        for j, col_j in enumerate(columns):
            # El Algoritmo PC usa 1 para i -> j y 3 para i <- j (y 2 para i -- j)
            if adj_matrix[i, j] == 1:
                # i -> j
                G.add_edge(col_i, col_j)
                if causal_strengths and (col_i, col_j) in causal_strengths:
                    weight = causal_strengths[(col_i, col_j)]
                    edge_labels[(col_i, col_j)] = f"{weight:.4f}"
            elif adj_matrix[i, j] == 2 and adj_matrix[j, i] == 2:
                # i -- j (Borde incierto, bidireccional para visualización)
                if not G.has_edge(col_j, col_i):  # Evitar duplicados
                    G.add_edge(col_i, col_j, weight='?')
    
    # Configuración de la visualización
    plt.figure(figsize=(12, 7))
    
    # Intentar usar Graphviz, pero con mejor manejo de excepciones
    pos = None
    try:
        pos = nx.drawing.nx_pydot.graphviz_layout(G, prog='dot')
        print("✓ Layout con Graphviz (dot) exitoso.")
    except Exception as e:
        print(f"⚠️  No se pudo usar Graphviz: {type(e).__name__}. Usando spring_layout...")
        pos = nx.spring_layout(G, k=2.5, iterations=100, seed=42)
    
    nx.draw_networkx_nodes(G, pos, node_color='lightcoral', node_size=1500, edgecolors='gray')
    
    # Dibuja aristas dirigidas (flechas)
    directed_edges = [(u, v) for u, v in G.edges() if G.edges[u, v].get('weight') != '?']
    nx.draw_networkx_edges(G, pos, edgelist=directed_edges, arrowstyle='->', 
                          arrowsize=20, edge_color='black', width=1.5)
    
    # Dibuja aristas inciertas (baja opacidad)
    undirected_edges = [(u, v) for u, v in G.edges() if G.edges[u, v].get('weight') == '?']
    nx.draw_networkx_edges(G, pos, edgelist=undirected_edges, arrowstyle='-', 
                          style='dashed', edge_color='gray', width=1.0, alpha=0.5)
    
    nx.draw_networkx_labels(G, pos, font_size=9, font_weight='bold')
    
    # Dibuja las etiquetas de fuerza causal
    if edge_labels:
        nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, 
                                    font_color='red', font_size=8, label_pos=0.5)
    
    plt.title(title, fontsize=14, fontweight='bold')
    plt.axis('off')
    plt.tight_layout()
    plt.show()

4. Bloque de ejecución principal

In [ ]:

# --- 1. Carga, Filtrado y Preprocesamiento de Datos ---
print("--- 1. Carga, Filtrado y Preprocesamiento de Datos ---")
try:
    data = pd.read_csv(FILE_PATH)
    data = data[SELECTED_COLUMNS]
    data = data.dropna()
    
    for col in data.columns:
        data[col] = pd.to_numeric(data[col], errors='coerce')
    
    data = data.dropna().astype(float)
    print(f"✅ Datos cargados y filtrados. Dimensiones finales: {data.shape}")
    print(f"   Columnas: {list(data.columns)}")
    
except FileNotFoundError:
    print(f"❌ ERROR: Archivo no encontrado en {FILE_PATH}")
    exit()
except Exception as e:
    print(f"❌ ERROR durante la carga o limpieza de datos: {e}")
    exit()

# --- 2. Estandarización de Datos ---
scaler = StandardScaler()
data_scaled_np = scaler.fit_transform(data)
data_scaled = pd.DataFrame(data_scaled_np, columns=data.columns)
print("\n[Estandarización de Datos]")
print("✅ Datos escalados con StandardScaler.")

# --- 3. DESCUBRIMIENTO CAUSAL (Algoritmo PC) ---
print("\n--- 3. Descubrimiento Causal (Algoritmo PC) ---")
try:
    data_matrix = data_scaled.values
    
    # Ejecutar el Algoritmo PC
    graph_pc = pc(data_matrix, alpha=0.05, indep_test=fisherz)
    
    # CORRECCIÓN: Acceso correcto a la matriz de adyacencia
    adj_matrix = graph_pc.G.graph
    
    print(f"✅ Descubrimiento de Estructura exitoso con Algoritmo PC.")
    print(f"   Variables analizadas: {len(data.columns)}")
    print(f"\nMatriz de Adyacencia (forma: {adj_matrix.shape}):")
    print(adj_matrix)
    
except Exception as e:
    print(f"\n❌ ERROR CRÍTICO en el Descubrimiento Causal PC: {e}")
    import traceback
    traceback.print_exc()
    exit()

# --- 4. CÁLCULO DE FUERZA CAUSAL (Regresión OLS) ---
print("\n--- 4. CÁLCULO DE FUERZA CAUSAL (Regresión OLS) ---")
causal_strengths = {}

# Iterar sobre cada variable como TARGET
for target_name in data_scaled.columns:
    target_index = data_scaled.columns.get_loc(target_name)
    
    # Identificar las variables PARENTES (causas) con flechas dirigidas al TARGET
    parents_indices = [i for i, col in enumerate(data_scaled.columns) 
                        if adj_matrix[i, target_index] == 1]
    parent_names = [data_scaled.columns[i] for i in parents_indices]
    
    if parent_names:
        # Construir el modelo de regresión
        Y = data_scaled[target_name]
        X = data_scaled[parent_names]
        X = add_constant(X)
        
        try:
            model = OLS(Y, X).fit()
            print(f"\n📊 Modelo para la variable DEPENDIENTE: {target_name}")
            print(f"   R-squared: {model.rsquared:.4f}")
            
            # Recorrer los resultados para obtener los coeficientes
            for parent in parent_names:
                coefficient = model.params.get(parent, 0.0)
                p_value = model.pvalues.get(parent, 1.0)
                
                # Usar el coeficiente solo si es estadísticamente significativo
                if p_value < 0.05: 
                    causal_strengths[(parent, target_name)] = coefficient
                    print(f"  ✓ {parent} → {target_name} | CS: {coefficient:.4f} (p={p_value:.4f})")
                else:
                    print(f"  ✗ {parent} → {target_name} | CS: {coefficient:.4f} (p={p_value:.4f}) [NO SIG]")
                    
        except Exception as e:
            print(f"  ❌ Error en regresión para {target_name}: {e}")

if not causal_strengths:
    print("\n⚠️  ADVERTENCIA: No se encontraron relaciones causales significativas.")
else:
    print(f"\n✅ Total de relaciones causales significativas encontradas: {len(causal_strengths)}")


Visualización del DAG con Fuerza Causal

In [ ]:
print("\n--- 5. Visualización del Grafo Causal ---")
plot_pc_dag(
    adj_matrix, 
    data_scaled.columns.tolist(), 
    causal_strengths=causal_strengths,
    title=f'Grafo Causal PC + Fuerza OLS (Alpha=0.05)'
)